In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hnswlib
from sklearn.neighbors import NearestNeighbors
from heapq import heappush, heappop

In [2]:

# Path to your GloVe file (update this based on your downloaded version)
glove_path = '/Users/tanishqchaudhari/Desktop/DataScience Proj  datasets/Dataset/glove.6B.100d.txt'

# Load GloVe vectors
word_to_vec = {}
words = []
vectors = []

with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]  # First token is the word
        vector = np.array(values[1:], dtype=np.float32)  # Rest are vector values
        word_to_vec[word] = vector
        words.append(word)
        vectors.append(vector)

# Convert to numpy array
vectors = np.array(vectors, dtype=np.float32)
print(f"Loaded {len(words)} word vectors of dimension {vectors.shape[1]}")

Loaded 400000 word vectors of dimension 100


In [3]:
print(vectors.shape)

(400000, 100)


In [4]:
# import numpy as np
# from heapq import heappush, heappop
# import random
# import math
# import time
# from sklearn.decomposition import PCA

# class HNSWQueryTimePCA:
#     """
#     HNSW implementation with conditional PCA application during query time.
#     The index is built with standard HNSW, but searches can optionally use PCA
#     in the top layers to accelerate distance calculations.
#     """
#     def __init__(self, dim, max_elements, M=16, ef_construction=200,
#                  pca_yes=False, pca_top_layers=3, pca_components=50):
#         """
#         Args:
#             dim (int): Dimensionality of vectors.
#             max_elements (int): Maximum expected elements (informational).
#             M (int): Maximum connections per node per layer.
#             ef_construction (int): Size of candidate list during construction.
#             pca_yes (bool): Whether to use PCA during query time.
#             pca_top_layers (int): Number of top layers to apply PCA if pca_yes=True.
#             pca_components (int): Target dimensionality after PCA reduction.
#         """
#         self.dim = dim
#         self.max_elements = max_elements
#         self.M = M
#         self.ef_construction = ef_construction
        
#         # PCA parameters
#         self.pca_yes = pca_yes
#         self.pca_top_layers = pca_top_layers
#         self.pca_components = min(pca_components, dim)  # Can't exceed original dimension
        
#         # HNSW data structures
#         self.layers = []  # Store graph layers: layers[l] is a dict {node_idx: [neighbor_indices]}
#         self.vectors = []  # Store the actual vectors
#         self.entry_point = None  # Entry point for the graph
        
#         # PCA data - only used if pca_yes=True
#         self.pca_models = {}  # {layer_idx: pca_model}
#         self.reduced_vectors = {}  # {node_idx: reduced_vector}
#         self.pca_applied_layers = set()  # Layers where PCA is active
        
#         print(f"Initialized HNSW with {'PCA enabled' if pca_yes else 'standard'} querying")

#     def _get_layer(self):
#         """Determine the layer for a new node based on exponential decay."""
#         ml = 1 / math.log(self.M) if self.M > 1 else 1
#         return max(0, int(-math.log(random.random()) * ml))

#     def _distance(self, idx1, idx2):
#         """Compute Euclidean distance between vectors using their indices."""
#         if not (0 <= idx1 < len(self.vectors) and 0 <= idx2 < len(self.vectors)):
#             return float('inf')
#         return np.linalg.norm(self.vectors[idx1] - self.vectors[idx2])

#     def _search_layer_standard(self, query_vec, layer_idx, ep_idx, ef):
#         """Standard HNSW layer search without PCA."""
#         layer_graph = self.layers[layer_idx]
#         if not layer_graph: return []
#         if ep_idx not in layer_graph:
#             try: ep_idx = next(iter(layer_graph))
#             except StopIteration: return []

#         visited = set()
#         candidates = []  # Min-heap: (dist, idx)
#         results = []     # Max-heap: (-dist, idx)

#         initial_dist = np.linalg.norm(query_vec - self.vectors[ep_idx])
#         heappush(candidates, (initial_dist, ep_idx))
#         heappush(results, (-initial_dist, ep_idx))
#         visited.add(ep_idx)

#         while candidates:
#             dist_candidate, current_idx = heappop(candidates)
#             farthest_dist_in_results = -results[0][0] if results else float('inf')

#             if dist_candidate > farthest_dist_in_results and len(results) >= ef:
#                 break

#             for neighbor_idx in layer_graph.get(current_idx, []):
#                 if neighbor_idx not in visited and neighbor_idx < len(self.vectors):
#                     visited.add(neighbor_idx)
#                     dist_neighbor = np.linalg.norm(query_vec - self.vectors[neighbor_idx])
#                     farthest_dist_in_results = -results[0][0] if results else float('inf')

#                     if dist_neighbor < farthest_dist_in_results or len(results) < ef:
#                         heappush(results, (-dist_neighbor, neighbor_idx))
#                         if len(results) > ef: heappop(results)
#                         heappush(candidates, (dist_neighbor, neighbor_idx))

#         final_results = sorted([(-d, idx) for d, idx in results])
#         return final_results[:ef]

#     def _search_layer_pca(self, query_vec, layer_idx, ep_idx, ef):
#         """PCA-enhanced layer search for top layers."""
#         layer_graph = self.layers[layer_idx]
#         if not layer_graph: return []
#         if ep_idx not in layer_graph:
#             try: ep_idx = next(iter(layer_graph))
#             except StopIteration: return []

#         visited = set()
#         candidates = []  # Min-heap: (dist, idx)
#         results = []     # Max-heap: (-dist, idx)

#         # Determine if we're in a PCA layer and transform query if needed
#         is_pca_layer = layer_idx in self.pca_applied_layers
#         query_vec_transformed = None
#         if is_pca_layer:
#             pca_model = self.pca_models.get(layer_idx)
#             if pca_model:
#                 try:
#                     query_vec_transformed = pca_model.transform(query_vec.reshape(1, -1))[0]
#                 except Exception as e:
#                     is_pca_layer = False  # Fall back to standard if transform fails

#         # Calculate initial distance (PCA space if applicable)
#         initial_dist = float('inf')
#         if is_pca_layer and ep_idx in self.reduced_vectors:
#             initial_dist = np.linalg.norm(query_vec_transformed - self.reduced_vectors[ep_idx])
#         if initial_dist == float('inf'):  # Fallback to original
#             initial_dist = np.linalg.norm(query_vec - self.vectors[ep_idx])

#         heappush(candidates, (initial_dist, ep_idx))
#         heappush(results, (-initial_dist, ep_idx))
#         visited.add(ep_idx)

#         while candidates:
#             dist_candidate, current_idx = heappop(candidates)
#             farthest_dist_in_results = -results[0][0] if results else float('inf')

#             if dist_candidate > farthest_dist_in_results and len(results) >= ef:
#                 break

#             for neighbor_idx in layer_graph.get(current_idx, []):
#                 if neighbor_idx not in visited and neighbor_idx < len(self.vectors):
#                     visited.add(neighbor_idx)
                    
#                     # PCA distance if applicable
#                     dist_neighbor = float('inf')
#                     if is_pca_layer and neighbor_idx in self.reduced_vectors:
#                         dist_neighbor = np.linalg.norm(query_vec_transformed - self.reduced_vectors[neighbor_idx])
#                     if dist_neighbor == float('inf'):  # Fallback
#                         dist_neighbor = np.linalg.norm(query_vec - self.vectors[neighbor_idx])
                        
#                     farthest_dist_in_results = -results[0][0] if results else float('inf')

#                     if dist_neighbor < farthest_dist_in_results or len(results) < ef:
#                         heappush(results, (-dist_neighbor, neighbor_idx))
#                         if len(results) > ef: heappop(results)
#                         heappush(candidates, (dist_neighbor, neighbor_idx))

#         final_results = sorted([(-d, idx) for d, idx in results])
#         return final_results[:ef]

#     def _search_layer(self, query_vec, layer_idx, ep_idx, ef):
#         """Unified search layer method that delegates to the appropriate implementation."""
#         if self.pca_yes and layer_idx in self.pca_applied_layers:
#             return self._search_layer_pca(query_vec, layer_idx, ep_idx, ef)
#         else:
#             return self._search_layer_standard(query_vec, layer_idx, ep_idx, ef)

#     def insert(self, vector):
#         """Insert a vector into the HNSW index (standard implementation)."""
#         new_idx = len(self.vectors)
#         self.vectors.append(np.array(vector))
#         node_max_layer = self._get_layer()
#         current_ep_idx = self.entry_point

#         while len(self.layers) <= node_max_layer:
#             self.layers.append(dict())

#         # Find entry points for insertion phase
#         for l in range(len(self.layers) - 1, node_max_layer, -1):
#             if current_ep_idx is None: break
#             if not self.layers[l]: continue
#             search_results = self._search_layer_standard(vector, l, current_ep_idx, ef=1)
#             if search_results:
#                 current_ep_idx = search_results[0][1]

#         # Insertion phase
#         for l in range(min(node_max_layer, len(self.layers) - 1), -1, -1):
#             layer_graph = self.layers[l]
#             if current_ep_idx is None and layer_graph:
#                 try:
#                     current_ep_idx = next(iter(layer_graph))
#                 except StopIteration:
#                     pass

#             if current_ep_idx is None:
#                 neighbors = []
#             else:
#                 neighbors = self._search_layer_standard(vector, l, current_ep_idx, self.ef_construction)
#                 if neighbors:
#                     current_ep_idx = neighbors[0][1]

#             connections = [idx for dist, idx in neighbors[:self.M]]
#             layer_graph[new_idx] = connections

#             # Add bidirectional links
#             for neighbor_idx in connections:
#                 if neighbor_idx not in layer_graph: 
#                     layer_graph[neighbor_idx] = []
                
#                 neighbor_connections = layer_graph[neighbor_idx]
#                 if new_idx not in neighbor_connections:
#                     neighbor_connections.append(new_idx)
#                     if len(neighbor_connections) > self.M:
#                         distances = [(self._distance(neighbor_idx, conn_idx), conn_idx) 
#                                     for conn_idx in neighbor_connections]
#                         distances.sort()
#                         layer_graph[neighbor_idx] = [idx for dist, idx in distances[:self.M]]

#         # Update global entry point
#         if node_max_layer >= len(self.layers) - 1 or self.entry_point is None:
#             self.entry_point = new_idx

#     def finalize_pca(self):
#         """
#         Prepares PCA models for the top layers after all vectors are inserted.
#         Only needed if pca_yes=True.
#         """
#         if not self.pca_yes:
#             print("PCA is disabled (pca_yes=False). Skipping PCA finalization.")
#             return
            
#         print(f"Applying PCA to top {self.pca_top_layers} layers (target dim: {self.pca_components})...")
#         num_actual_layers = len(self.layers)
#         if num_actual_layers == 0:
#             print("No layers found, skipping PCA.")
#             return

#         # Determine which layers to apply PCA to
#         start_pca_layer = max(0, num_actual_layers - self.pca_top_layers)

#         for l_idx in range(start_pca_layer, num_actual_layers):
#             layer_graph = self.layers[l_idx]
#             node_indices = list(layer_graph.keys())

#             if not node_indices:
#                 print(f"Layer {l_idx} is empty, skipping PCA.")
#                 continue

#             valid_indices = [idx for idx in node_indices if idx < len(self.vectors)]
#             if len(valid_indices) <= self.pca_components:
#                 print(f"Layer {l_idx} has only {len(valid_indices)} nodes, fewer than required {self.pca_components} for PCA.")
#                 continue

#             try:
#                 layer_vectors = np.array([self.vectors[idx] for idx in valid_indices])
#                 print(f"Fitting PCA for layer {l_idx} on {len(valid_indices)} vectors...")
#                 pca = PCA(n_components=self.pca_components)
#                 pca.fit(layer_vectors)
#                 reduced_vectors = pca.transform(layer_vectors)
                
#                 self.pca_models[l_idx] = pca
#                 self.pca_applied_layers.add(l_idx)
#                 for idx, reduced_vec in zip(valid_indices, reduced_vectors):
#                     self.reduced_vectors[idx] = reduced_vec
#                 print(f"PCA applied successfully to layer {l_idx}")
#             except Exception as e:
#                 print(f"Error applying PCA to layer {l_idx}: {e}")
#                 continue

#         print(f"PCA finalization complete. Applied to {len(self.pca_applied_layers)} layers.")

#     def search(self, query_vec, k=10):
#         """Search for the top-k nearest neighbors using standard or PCA-enhanced search."""
#         query_vec = np.array(query_vec)
#         ef_search = max(self.ef_construction, k)
#         current_ep_idx = self.entry_point

#         if current_ep_idx is None:
#             return []

#         # Search down from top layer to find entry point for base layer
#         for l in range(len(self.layers) - 1, 0, -1):
#             if not self.layers[l]: continue
#             search_results = self._search_layer(query_vec, l, current_ep_idx, ef=1)
#             if search_results:
#                 current_ep_idx = search_results[0][1]

#         # Search in base layer
#         neighbors = self._search_layer(query_vec, 0, current_ep_idx, ef_search)
#         return [idx for dist, idx in neighbors[:k]]


In [5]:
# def compare_standard_vs_pca_hnsw(vectors, num_queries=1000, k=10, M=16, ef_construction=200, 
#                                ef_search=200, pca_top_layers=3, pca_components=50):
#     """
#     Builds one HNSW index and compares standard vs PCA-enabled query performance.
    
#     Args:
#         vectors: np.ndarray of shape (n, dim) containing the vectors to index
#         num_queries: number of queries to run (first num_queries vectors will be used)
#         k: number of nearest neighbors to retrieve
#         M: HNSW M parameter (max connections per node)
#         ef_construction: HNSW ef_construction parameter
#         ef_search: HNSW ef_search parameter for querying
#         pca_top_layers: number of top layers to apply PCA to
#         pca_components: target dimensionality after PCA reduction
    
#     Returns:
#         dict: Results comparison
#     """
#     import time
#     from sklearn.neighbors import NearestNeighbors
    
#     N, dim = vectors.shape
#     # Get total number of vectors
#     N = vectors.shape[0]  # Note: Use vectors.shape[0] to get count, not just vectors.shape

#     # Choose 1000 random indices without replacement
#     num_queries = 1000
#     random_indices = np.random.choice(N, size=min(num_queries, N), replace=False)

#     # Select the corresponding vectors
#     query_vectors = vectors[random_indices]
    
    
#     # Calculate ground truth
#     print(f"Calculating ground truth for {len(query_vectors)} queries (k={k})...")
#     start_time = time.perf_counter()
#     bf_index = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
#     bf_index.fit(vectors)
#     _, ground_truth_indices = bf_index.kneighbors(query_vectors)
#     gt_time = time.perf_counter() - start_time
#     print(f"Ground truth calculated in {gt_time:.4f} s.")
    
#     # Helper function for recall calculation
#     def calculate_recall(ground_truth, found_neighbors, k):
#         total_found = 0
#         for i in range(len(ground_truth)):
#             gt_set = set(ground_truth[i][:k])
#             found_set = set(found_neighbors[i][:k])
#             total_found += len(gt_set.intersection(found_set))
#         return total_found / (len(ground_truth) * k)
    
#     # Build index with PCA disabled initially
#     print("\n--- Building HNSW index ---")
#     index = HNSWQueryTimePCA(dim=dim, max_elements=N, M=M, ef_construction=ef_construction,
#                              pca_yes=False, pca_top_layers=pca_top_layers, 
#                              pca_components=pca_components)
    
#     # Insert all vectors
#     start_time = time.perf_counter()
#     for i in range(N):
#         index.insert(vectors[i])
#     build_time = time.perf_counter() - start_time
#     print(f"Index built in {build_time:.4f} s")
    
#     # 1. Standard querying (pca_yes=False)
#     print("\n--- Standard HNSW Querying ---")
#     index.pca_yes = False
#     standard_results = []
    
#     start_time = time.perf_counter()
#     for i in range(len(query_vectors)):
#         results = index.search(query_vectors[i], k=k)
#         standard_results.append(results)
#     standard_query_time = time.perf_counter() - start_time
#     standard_avg_time = standard_query_time / len(query_vectors)
    
#     standard_recall = calculate_recall(ground_truth_indices, standard_results, k)
#     print(f"Standard Query time: {standard_query_time:.4f} s ({standard_avg_time*1000:.4f} ms/query)")
#     print(f"Standard Recall@{k}: {standard_recall:.4f}")
    
#     # 2. Prepare and run PCA-enabled querying
#     print("\n--- PCA-Enhanced HNSW Querying ---")
#     index.pca_yes = True
#     index.finalize_pca()  # Train PCA models
    
#     pca_results = []
#     start_time = time.perf_counter()
#     for i in range(len(query_vectors)):
#         results = index.search(query_vectors[i], k=k)
#         pca_results.append(results)
#     pca_query_time = time.perf_counter() - start_time
#     pca_avg_time = pca_query_time / len(query_vectors)
    
#     pca_recall = calculate_recall(ground_truth_indices, pca_results, k)
#     print(f"PCA Query time: {pca_query_time:.4f} s ({pca_avg_time*1000:.4f} ms/query)")
#     print(f"PCA Recall@{k}: {pca_recall:.4f}")
    
#     # Summary
#     print("\n--- Comparison Summary ---")
#     print(f"Method              | Recall@{k}  | Avg Query Time (ms)")
#     print(f"--------------------|------------|-------------------")
#     print(f"Standard HNSW       | {standard_recall:.4f}     | {standard_avg_time*1000:.4f}")
#     print(f"PCA-Enhanced HNSW   | {pca_recall:.4f}     | {pca_avg_time*1000:.4f}")
    
#     return {
#         'standard_recall': standard_recall,
#         'standard_query_time': standard_query_time,
#         'pca_recall': pca_recall,
#         'pca_query_time': pca_query_time,
#         'build_time': build_time
#     }


In [6]:
# # Assuming 'vectors' is your data (e.g., GloVe vectors)
# # vectors: np.ndarray of shape (N, dim)

# # Run comparison
# results = compare_standard_vs_pca_hnsw(
#     vectors=vectors,
#     num_queries=1000,  # Number of queries to run
#     k=10,              # Number of neighbors to retrieve
#     M=16,              # HNSW parameter
#     ef_construction=200,  # Index build quality parameter
#     ef_search=200,     # Search quality parameter
#     pca_top_layers=3,  # Apply PCA to top 3 layers
#     pca_components=50  # Reduce to 50 dimensions in those layers
# )


In [7]:
import numpy as np
import time
import logging
import random
import math
from heapq import heappush, heappop
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [8]:
# def precompute_all_query_pcas(pca_models, pca_applied_layers, query_vectors):
#     """
#     Precomputes PCA projections for a batch of query vectors.
#     Returns dict {layer_idx: array(shape=(n_queries, pca_components))}.
#     """
#     pca_queries = {}
#     query_vectors_np = np.asarray(query_vectors)

#     if not pca_models or not pca_applied_layers:
#         logger.warning("No PCA models or applied layers provided; skipping precomputation.")
#         return pca_queries

#     for l_idx in pca_applied_layers:
#         model = pca_models.get(l_idx)
#         if model is None:
#             logger.warning(f"PCA model for layer {l_idx} not found; skipping.")
#             continue
#         pca_queries[l_idx] = model.transform(query_vectors_np)
#         logger.debug(f"Layer {l_idx}: precomputed PCA queries shape {pca_queries[l_idx].shape}")

#     return pca_queries

In [9]:
import numpy as np

def precompute_all_query_pcas(pca_models, pca_applied_layers, query_vectors):
    """
    Precomputes PCA projections for a batch of query vectors.
    Returns a list where each index corresponds to the layer number, and contains
    either the PCA-transformed queries (np.ndarray) or None if not used.
    """
    max_layer = max(pca_applied_layers) + 1 if pca_applied_layers else 0
    pca_queries = [None] * max_layer  # indexed list for O(1) access

    query_vectors_np = np.asarray(query_vectors)

    for l_idx in pca_applied_layers:
        model = pca_models[l_idx]  # direct index, avoid .get()
        pca_queries[l_idx] = model.transform(query_vectors_np)

    return pca_queries


In [10]:
# class optHNSWPCA:
#     """
#     HNSW with optional PCA-based acceleration.
#     """
#     def __init__(self, dim, max_elements, M=16, ef_construction=200,
#                  pca_yes=False, pca_top_layers=3, pca_components=50):
#         self.dim = dim
#         self.max_elements = max_elements
#         self.M = M
#         self.ef_construction = ef_construction
#         self.pca_yes = pca_yes and pca_components < dim
#         self.pca_top_layers = pca_top_layers
#         self.pca_components = min(pca_components, dim)

#         self.vectors = []
#         self.layers = []
#         self.entry_point = None

#         # will populate after finalize_pca()
#         self.pca_models = {}
#         self.reduced_vectors = {}
#         self.pca_applied_layers = set()

#         logger.info(f"Initialized HNSW (dim={dim}, PCA={'on' if self.pca_yes else 'off'})")

#     def _get_layer(self):
#         ml = 1 / math.log(self.M) if self.M > 1 else 1
#         level = int(-math.log(random.random()) * ml) if self.M > 1 else 0
#         return max(0, level)

#     def _distance(self, idx1, idx2):
#         return np.linalg.norm(self.vectors[idx1] - self.vectors[idx2])

#     def _search_layer_standard(self, query_vec, layer_idx, ep_idx, ef):
#         """Standard full-dimensional search on one layer."""
#         graph = self.layers[layer_idx]
#         if not graph:
#             return []
#         if ep_idx not in graph:
#             ep_idx = next(iter(graph))

#         visited = {ep_idx}
#         candidates = [(np.linalg.norm(query_vec - self.vectors[ep_idx]), ep_idx)]
#         results = [(-candidates[0][0], ep_idx)]

#         while candidates:
#             dist, cur = heappop(candidates)
#             farthest = -results[0][0]
#             if dist > farthest and len(results) >= ef:
#                 break

#             for nb in graph[cur]:
#                 if nb in visited: continue
#                 visited.add(nb)
#                 d = np.linalg.norm(query_vec - self.vectors[nb])
#                 farthest = -results[0][0]
#                 if d < farthest or len(results) < ef:
#                     heappush(results, (-d, nb))
#                     if len(results) > ef:
#                         heappop(results)
#                     heappush(candidates, (d, nb))

#         return sorted([(-d, idx) for d, idx in results])[:ef]

#     def _search_layer_pca_precomputed(self, query_idx, pca_queries, layer_idx, ep_idx, ef):
#         """PCA-based search using precomputed query vectors."""
#         graph = self.layers[layer_idx]
#         if not graph:
#             return []
#         if ep_idx not in graph:
#             ep_idx = next(iter(graph))

#         qvec = pca_queries[layer_idx][query_idx]
#         visited = {ep_idx}
#         rv = self.reduced_vectors
#         init = np.linalg.norm(qvec - rv[ep_idx])
#         candidates = [(init, ep_idx)]
#         results = [(-init, ep_idx)]

#         while candidates:
#             dist, cur = heappop(candidates)
#             farthest = -results[0][0]
#             if dist > farthest and len(results) >= ef:
#                 break

#             for nb in graph[cur]:
#                 if nb in visited: continue
#                 visited.add(nb)
#                 d = np.linalg.norm(qvec - rv.get(nb, qvec * 0))
#                 farthest = -results[0][0]
#                 if d < farthest or len(results) < ef:
#                     heappush(results, (-d, nb))
#                     if len(results) > ef:
#                         heappop(results)
#                     heappush(candidates, (d, nb))

#         return sorted([(-d, idx) for d, idx in results])[:ef]

#     def insert(self, vector):
#         vec = np.asarray(vector)
#         idx = len(self.vectors)
#         if idx >= self.max_elements:
#             raise MemoryError("Index full")
#         self.vectors.append(vec)
#         level = self._get_layer()
#         while len(self.layers) <= level:
#             self.layers.append({})

#         ep = self.entry_point
#         # search down from top
#         for l in range(len(self.layers)-1, level, -1):
#             if ep is None or not self.layers[l]: break
#             res = self._search_layer_standard(vec, l, ep, ef=1)
#             if res:
#                 ep = res[0][1]

#         for l in range(level, -1, -1):
#             graph = self.layers[l]
#             if ep is None and graph:
#                 ep = next(iter(graph))

#             neigh = []
#             if ep is not None:
#                 neigh = self._search_layer_standard(vec, l, ep, self.ef_construction)
#                 if neigh:
#                     ep = neigh[0][1]
#             conns = [i for _, i in neigh[:self.M]]
#             graph[idx] = conns
#             for nb in conns:
#                 graph.setdefault(nb, []).append(idx)
#                 if len(graph[nb]) > self.M:
#                     # prune
#                     dists = [(self._distance(nb, c), c) for c in graph[nb]]
#                     dists.sort()
#                     graph[nb] = [c for _, c in dists[:self.M]]

#         if self.entry_point is None or level > self._get_node_layer(self.entry_point):
#             self.entry_point = idx

#     def finalize_pca(self):
#         if not self.pca_yes:
#             logger.info("PCA disabled or no reduction needed.")
#             return

#         layers = self.layers
#         total = len(layers)
#         start = max(0, total - self.pca_top_layers)
#         self.pca_models.clear()
#         self.reduced_vectors.clear()
#         self.pca_applied_layers.clear()

#         for l in range(start, total):
#             graph = layers[l]
#             idxs = [i for i in graph if i < len(self.vectors)]
#             if len(idxs) <= self.pca_components:
#                 continue
#             arr = np.vstack([self.vectors[i] for i in idxs])
#             pca = PCA(n_components=self.pca_components)
#             pca.fit(arr)
#             red = pca.transform(arr)
#             self.pca_models[l] = pca
#             self.pca_applied_layers.add(l)
#             for i_node, vec_red in zip(idxs, red):
#                 self.reduced_vectors[i_node] = vec_red
#         logger.info(f"PCA applied to layers {sorted(self.pca_applied_layers)}")

#     def search(self, query_vec, k=10):
#         ef = max(self.ef_construction, k)
#         ep = self.entry_point
#         if ep is None:
#             return []
#         q = np.asarray(query_vec)

#         # top-down traversal
#         for l in range(len(self.layers)-1, 0, -1):
#             if not self.layers[l]: continue
#             res = self._search_layer_standard(q, l, ep, ef=1)
#             if res:
#                 ep = res[0][1]

#         # ground search
#         res = self._search_layer_standard(q, 0, ep, ef)
#         return [i for _, i in res[:k]]

#     def search_pca_opt(self, query_idx, query_vec, pca_queries, k=10):
#         ef = max(self.ef_construction, k)
#         ep = self.entry_point
#         if ep is None:
#             return []

#         # top-down traversal with PCA where available
#         for l in range(len(self.layers)-1, 0, -1):
#             if not self.layers[l]: continue
#             if self.pca_yes and l in self.pca_applied_layers and l in pca_queries:
#                 res = self._search_layer_pca_precomputed(query_idx, pca_queries, l, ep, ef=1)
#             else:
#                 # fallback to full-dim search
#                 res = self._search_layer_standard(query_vec, l, ep, ef=1)
#             if res:
#                 ep = res[0][1]

#         # base layer
#         if 0 in self.pca_applied_layers and 0 in pca_queries and self.pca_yes:
#             res = self._search_layer_pca_precomputed(query_idx, pca_queries, 0, ep, ef)
#         else:
#             res = self._search_layer_standard(query_vec, 0, ep, ef)

#         return [i for _, i in res[:k]]

#     def _get_node_layer(self, node_idx):
#         return max((l for l, g in enumerate(self.layers) if node_idx in g), default=-1)



In [11]:
def precompute_all_query_pcas(pca_models, pca_applied_layers, query_vectors):
    """
    Precomputes PCA projections for a batch of query vectors.
    Returns a list where each index = layer number, and value is
    either the (n_queries, pca_components) array or None.
    """
    if not pca_applied_layers:
        return []

    max_layer = max(pca_applied_layers) + 1
    pca_queries = [None] * max_layer
    Q = np.asarray(query_vectors)

    for l in pca_applied_layers:
        model = pca_models[l]
        pca_queries[l] = model.transform(Q)

    return pca_queries

In [12]:
import math
import random
import logging
import numpy as np
from sklearn.decomposition import PCA
from heapq import heappush, heappop

logger = logging.getLogger(__name__)

def precompute_all_query_pcas(pca_models, pca_applied_layers, query_vectors):
    """
    Precomputes PCA projections for a batch of query vectors.
    Returns a list where each index = layer number, and value is
    either the (n_queries, pca_components) array or None.
    """
    if not pca_applied_layers:
        return []

    max_layer = max(pca_applied_layers) + 1
    pca_queries = [None] * max_layer
    Q = np.asarray(query_vectors)

    for l in pca_applied_layers:
        model = pca_models[l]
        pca_queries[l] = model.transform(Q)

    return pca_queries

class optHNSWPCA:
    """
    HNSW with optional PCA-based acceleration.
    """
    def __init__(self, dim, max_elements, M=16, ef_construction=200,
                 pca_yes=False, pca_top_layers=3, pca_components=50):
        self.dim = dim
        self.max_elements = max_elements
        self.M = M
        self.ef_construction = ef_construction
        self.pca_yes = pca_yes and pca_components < dim
        self.pca_top_layers = pca_top_layers
        self.pca_components = min(pca_components, dim)

        self.vectors = []
        self.layers = []
        self.entry_point = None
        self.entry_point_level = -1

        # PCA internals
        self.pca_models = {}
        self.pca_applied_layers = set()

        # optimized lookup structures
        self._rv_arr = None      # shape (max_elements, pca_components)
        self._rv_mask = None     # boolean mask per node index
        self._use_pca_layer = None  # boolean mask per layer index

        logger.info(f"Initialized HNSW (dim={dim}, PCA={'on' if self.pca_yes else 'off'})")

    def _get_layer(self):
        ml = 1 / math.log(self.M) if self.M > 1 else 1
        return max(0, int(-math.log(random.random()) * ml))

    def _distance(self, idx1, idx2):
        return np.linalg.norm(self.vectors[idx1] - self.vectors[idx2])

    def _search_layer_standard(self, query_vec, layer_idx, ep_idx, ef):
        graph = self.layers[layer_idx]
        if not graph:
            return []
        if ep_idx not in graph:
            ep_idx = next(iter(graph))

        visited = {ep_idx}
        dist0 = np.linalg.norm(query_vec - self.vectors[ep_idx])
        candidates = [(dist0, ep_idx)]
        results = [(-dist0, ep_idx)]

        while candidates:
            dist, cur = heappop(candidates)
            farthest = -results[0][0]
            if dist > farthest and len(results) >= ef:
                break
            for nb in graph[cur]:
                if nb in visited:
                    continue
                visited.add(nb)
                d = np.linalg.norm(query_vec - self.vectors[nb])
                if d < farthest or len(results) < ef:
                    heappush(results, (-d, nb))
                    if len(results) > ef:
                        heappop(results)
                    heappush(candidates, (d, nb))

        return sorted([(-d, idx) for d, idx in results])[:ef]

    def insert(self, vector):
        vec = np.asarray(vector)
        idx = len(self.vectors)
        if idx >= self.max_elements:
            raise MemoryError("Index full")
        self.vectors.append(vec)
        level = self._get_layer()
        while len(self.layers) <= level:
            self.layers.append({})

        ep = self.entry_point
        # search down from top layers to 'level'
        for l in range(len(self.layers) - 1, level, -1):
            if ep is None or not self.layers[l]:
                break
            res = self._search_layer_standard(vec, l, ep, ef=1)
            if res:
                ep = res[0][1]

        # connect this node on each layer up to 'level'
        for l in range(level, -1, -1):
            graph = self.layers[l]
            if ep is None and graph:
                ep = next(iter(graph))
            neigh = []
            if ep is not None:
                neigh = self._search_layer_standard(vec, l, ep, self.ef_construction)
                if neigh:
                    ep = neigh[0][1]

            conns = [nid for _, nid in neigh[:self.M]]
            graph[idx] = conns
            for nb in conns:
                graph.setdefault(nb, []).append(idx)
                if len(graph[nb]) > self.M:
                    dists = [(self._distance(nb, c), c) for c in graph[nb]]
                    dists.sort()
                    graph[nb] = [c for _, c in dists[:self.M]]

        # update entry point based on known 'level'
        if self.entry_point is None or level > self.entry_point_level:
            self.entry_point = idx
            self.entry_point_level = level

    def finalize_pca(self):
        if not self.pca_yes:
            logger.info("PCA disabled or no reduction needed.")
            return

        total_layers = len(self.layers)
        start = max(0, total_layers - self.pca_top_layers)
        self.pca_models.clear()
        self.pca_applied_layers.clear()

        # fit PCA on each top layer
        for l in range(start, total_layers):
            idxs = [i for i in self.layers[l] if i < len(self.vectors)]
            if len(idxs) <= self.pca_components:
                continue
            arr = np.vstack([self.vectors[i] for i in idxs])
            pca = PCA(n_components=self.pca_components)
            pca.fit(arr)
            self.pca_models[l] = pca
            self.pca_applied_layers.add(l)

        # allocate optimized structures
        self._rv_arr = np.zeros((self.max_elements, self.pca_components), dtype=float)
        self._rv_mask = np.zeros(self.max_elements, dtype=bool)
        self._use_pca_layer = np.zeros(len(self.layers), dtype=bool)

        # fill reduced vectors & flags
        for l in self.pca_applied_layers:
            idxs = [i for i in self.layers[l] if i < len(self.vectors)]
            arr = np.vstack([self.vectors[i] for i in idxs])
            red = self.pca_models[l].transform(arr)
            for node, vec_red in zip(idxs, red):
                self._rv_arr[node] = vec_red
                self._rv_mask[node] = True
            self._use_pca_layer[l] = True

        logger.info(f"PCA applied to layers {sorted(self.pca_applied_layers)}")

    def _search_layer_pca_precomputed(self, query_idx, query_vec, pca_queries, layer_idx, ep_idx, ef):
        graph = self.layers[layer_idx]
        if not graph:
            return []
        q_red = pca_queries[layer_idx][query_idx]

        visited = {ep_idx}
        init = np.linalg.norm(q_red - self._rv_arr[ep_idx])
        candidates = [(init, ep_idx)]
        results = [(-init, ep_idx)]

        while candidates:
            dist, cur = heappop(candidates)
            farthest = -results[0][0]
            if dist > farthest and len(results) >= ef:
                break
            for nb in graph[cur]:
                if nb in visited:
                    continue
                visited.add(nb)
                if self._rv_mask[nb]:
                    d = np.linalg.norm(q_red - self._rv_arr[nb])
                else:
                    d = np.linalg.norm(query_vec - self.vectors[nb])
                if d < farthest or len(results) < ef:
                    heappush(results, (-d, nb))
                    if len(results) > ef:
                        heappop(results)
                    heappush(candidates, (d, nb))

        return sorted([(-d, idx) for d, idx in results])[:ef]

    def search(self, query_vec, k=10):
        ef = max(self.ef_construction, k)
        ep = self.entry_point
        if ep is None:
            return []
        q = np.asarray(query_vec)
        # top-down traversal
        for l in range(len(self.layers) - 1, 0, -1):
            if not self.layers[l]:
                continue
            res = self._search_layer_standard(q, l, ep, ef=1)
            if res:
                ep = res[0][1]
        # base layer search
        res = self._search_layer_standard(q, 0, ep, ef)
        return [i for _, i in res[:k]]

    def search_pca_opt(self, query_idx, query_vec, pca_queries, k=10):
        ef = max(self.ef_construction, k)
        ep = self.entry_point
        if ep is None:
            return []
        q = np.asarray(query_vec)

        # top-down traversal
        for l in range(len(self.layers) - 1, 0, -1):
            if not self.layers[l]:
                continue
            if self._use_pca_layer[l] and l < len(pca_queries) and pca_queries[l] is not None:
                res = self._search_layer_pca_precomputed(query_idx, q, pca_queries, l, ep, ef=1)
            else:
                res = self._search_layer_standard(q, l, ep, ef=1)
            if res:
                ep = res[0][1]

        # base layer search
        if self._use_pca_layer[0] and pca_queries and pca_queries[0] is not None:
            res = self._search_layer_pca_precomputed(query_idx, q, pca_queries, 0, ep, ef)
        else:
            res = self._search_layer_standard(q, 0, ep, ef)

        return [i for (_, i) in res[:k]]


In [13]:
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

# Ensure optHNSWPCA and precompute_all_query_pcas are in scope
# from your_module import optHNSWPCA, precompute_all_query_pcas

if __name__ == "__main__":
    # --- parameters ---
    num_vectors = 400_000
    num_queries = 40_000
    dim = 100
    k = 100
    M = 16
    ef_construction = 200
    pca_top_layers = 3
    pca_components = 20

    # load or generate data
    print("Generating data…")
    data = vectors
    queries = data[np.random.choice(num_vectors, num_queries, replace=False)]

    # compute ground truth once
    print("Computing ground truth…")
    nn = NearestNeighbors(n_neighbors=k, algorithm='brute')
    nn.fit(data)
    gt_d, gt_i = nn.kneighbors(queries)

    # --- Build standard (non-PCA) index ---
    print("Building standard HNSW index (no PCA)…")
    idx_std = optHNSWPCA(
        dim, num_vectors + 10, M, ef_construction,
        pca_yes=False, pca_top_layers=pca_top_layers,
        pca_components=pca_components
    )
    t0 = time.perf_counter()
    for v in tqdm(data, desc="Indexing standard HNSW"):
        idx_std.insert(v)
    build_std = time.perf_counter() - t0
    print(f"Standard build time: {build_std:.2f}s")

    # --- Build PCA-enabled index ---
    print("Building PCA-enabled HNSW index…")
    idx_pca = optHNSWPCA(
        dim, num_vectors + 10, M, ef_construction,
        pca_yes=True, pca_top_layers=pca_top_layers,
        pca_components=pca_components
    )
    t0 = time.perf_counter()
    for v in tqdm(data, desc="Indexing PCA-enabled HNSW"):
        idx_pca.insert(v)
    build_pca = time.perf_counter() - t0
    print(f"PCA-enabled build time: {build_pca:.2f}s")

    # finalize PCA on PCA-enabled index
    print("Finalizing PCA models on index…")
    t0 = time.perf_counter()
    idx_pca.finalize_pca()
    fit_pca = time.perf_counter() - t0
    print(f"PCA fit time: {fit_pca:.2f}s")

    # precompute PCA projections for queries
    pca_q = precompute_all_query_pcas(idx_pca.pca_models, idx_pca.pca_applied_layers, queries)

    # helper for recall
    def recall_at_k(truth, pred): return len(set(truth) & set(pred)) / k

    # evaluate standard search
    print("Evaluating standard search…")
    total_r_std, times_std = 0, []
    tstart = time.perf_counter()
    for i, q in enumerate(tqdm(queries, desc="Standard query")):
        ts = time.perf_counter()
        out = idx_std.search(q, k)
        te = time.perf_counter()
        times_std.append(te - ts)
        total_r_std += recall_at_k(gt_i[i], out)
    tot_time_std = time.perf_counter() - tstart
    avg_time_std = np.mean(times_std) * 1000
    recall_std = total_r_std / num_queries

    # evaluate PCA-optimized search
    print("Evaluating PCA-optimized search…")
    total_r_pca, times_pca = 0, []
    tstart = time.perf_counter()
    for i, q in enumerate(tqdm(queries, desc="PCA-opt query")):
        ts = time.perf_counter()
        out = idx_pca.search_pca_opt(i, q, pca_q, k)
        te = time.perf_counter()
        times_pca.append(te - ts)
        total_r_pca += recall_at_k(gt_i[i], out)
    tot_time_pca = time.perf_counter() - tstart
    avg_time_pca = np.mean(times_pca) * 1000
    recall_pca = total_r_pca / num_queries

    # output results
    print(f"Standard: recall={recall_std:.4f}, total_query_time={tot_time_std:.2f}s, avg_query_time={avg_time_std:.2f}ms")
    print(f"PCA-opt: recall={recall_pca:.4f}, total_query_time={tot_time_pca:.2f}s, avg_query_time={avg_time_pca:.2f}ms")

    print("\n--- Comparison Summary ---")
    print(f"Method         | Recall@{k} | TotalQuery(s) | AvgQuery(ms)")
    print(f"Standard       | {recall_std:.4f}    | {tot_time_std:.2f}       | {avg_time_std:.2f}")
    print(f"PCA-optimized  | {recall_pca:.4f}    | {tot_time_pca:.2f}       | {avg_time_pca:.2f}")


Generating data…
Computing ground truth…


INFO:__main__:Initialized HNSW (dim=100, PCA=off)


Building standard HNSW index (no PCA)…


Indexing standard HNSW: 100%|██████████| 400000/400000 [1:22:55<00:00, 80.40it/s]    
INFO:__main__:Initialized HNSW (dim=100, PCA=on)


Standard build time: 1437.76s
Building PCA-enabled HNSW index…


...

PCA fit time: 1.16s
Evaluating standard search…


Standard query: 100%|██████████| 5000/5000 [00:17<00:00, 293.20it/s]


Evaluating PCA-optimized search…


PCA-opt query: 100%|██████████| 5000/5000 [00:21<00:00, 233.55it/s]

Standard: recall=0.5113, total_query_time=17.06s, avg_query_time=3.37ms
PCA-opt: recall=0.1732, total_query_time=21.41s, avg_query_time=4.24ms

--- Comparison Summary ---
Method         | Recall@100 | TotalQuery(s) | AvgQuery(ms)
Standard       | 0.5113    | 17.06       | 3.37
PCA-optimized  | 0.1732    | 21.41       | 4.24


In [14]:
if __name__ == "__main__":
    

    # evaluate standard search
    print("Evaluating standard search…")
    total_r_std, times_std = 0, []
    tstart = time.perf_counter()
    for i, q in enumerate(tqdm(queries, desc="Standard query")):
        ts = time.perf_counter()
        out = idx_std.search(q, k)
        te = time.perf_counter()
        times_std.append(te - ts)
        total_r_std += recall_at_k(gt_i[i], out)
    tot_time_std = time.perf_counter() - tstart
    avg_time_std = np.mean(times_std) * 1000
    recall_std = total_r_std / num_queries

    # evaluate PCA-optimized search
    print("Evaluating PCA-optimized search…")
    total_r_pca, times_pca = 0, []
    tstart = time.perf_counter()
    for i, q in enumerate(tqdm(queries, desc="PCA-opt query")):
        ts = time.perf_counter()
        out = idx_pca.search_pca_opt(i, q, pca_q, k)
        te = time.perf_counter()
        times_pca.append(te - ts)
        total_r_pca += recall_at_k(gt_i[i], out)
    tot_time_pca = time.perf_counter() - tstart
    avg_time_pca = np.mean(times_pca) * 1000
    recall_pca = total_r_pca / num_queries

    # output results
    print(f"Standard: recall={recall_std:.4f}, total_query_time={tot_time_std:.2f}s, avg_query_time={avg_time_std:.2f}ms")
    print(f"PCA-opt: recall={recall_pca:.4f}, total_query_time={tot_time_pca:.2f}s, avg_query_time={avg_time_pca:.2f}ms")

    print("\n--- Comparison Summary ---")
    print(f"Method         | Recall@{k} | TotalQuery(s) | AvgQuery(ms)")
    print(f"Standard       | {recall_std:.4f}    | {tot_time_std:.2f}       | {avg_time_std:.2f}")
    print(f"PCA-optimized  | {recall_pca:.4f}    | {tot_time_pca:.2f}       | {avg_time_pca:.2f}")

Evaluating standard search…


Standard query: 100%|██████████| 5000/5000 [00:17<00:00, 291.77it/s]


Evaluating PCA-optimized search…


PCA-opt query: 100%|██████████| 5000/5000 [00:21<00:00, 233.50it/s]

Standard: recall=0.5113, total_query_time=17.14s, avg_query_time=3.39ms
PCA-opt: recall=0.1732, total_query_time=21.41s, avg_query_time=4.24ms

--- Comparison Summary ---
Method         | Recall@100 | TotalQuery(s) | AvgQuery(ms)
Standard       | 0.5113    | 17.14       | 3.39
PCA-optimized  | 0.1732    | 21.41       | 4.24


In [15]:
print("Height of HNSW graph:", idx_pca.entry_point_level)

Height of HNSW graph: 5
